In [1]:
import pandas as pd

PATH = "../data/dataset/"
DATASET = 'news_29_80k_clean_with_features_embs'

data = pd.read_pickle(f"{PATH}{DATASET}.pkl").dropna()
data["sign_count"] = data["sign_count"].apply(lambda x: int(x))

In [2]:
data

,text,genre,word_count,sign_count,smile_count,emb
0,вагнер может вернуться на фронт именно как ваг...,1,208.0,0,21.0,"[0.105845354, -0.009212951, -0.037567068, -0.0..."
1,киев заявляет о взятии клещеевки что под бахму...,1,225.0,0,7.0,"[0.0899639, 0.0006396173, -0.0027103138, -0.00..."
2,миграционное право самая отвратительная сфера ...,1,196.0,3,3.0,"[0.1243427, -0.073032424, -0.03801419, -0.0273..."
3,узнала что после череды прилетов по москвасити...,1,81.0,0,2.0,"[0.07559179, 0.029469201, 0.01854891, -0.02956..."
4,официальные власти ничего не говорят но вот пе...,1,113.0,0,2.0,"[0.072696425, 0.002974232, -0.019545708, -0.03..."
...,...,...,...,...,...,...
67961,лайфхак о котором вы даже не догадывались паре...,9,25.0,0,0.0,"[0.0052365544, -0.042056654, -0.0056621456, -0..."
67962,некоторым кто мечтал озвучивать сериалы очень ...,9,53.0,0,4.0,"[-0.013167946, -0.014489826, -0.016909203, -0...."
67963,джеймсу кэмерону предложили снять сериал о про...,9,38.0,0,1.0,"[-0.09007934, -0.02824813, -0.025535312, -0.07..."
67964,bloodborne портируют в minecraft энтузиаст под...,9,34.0,0,2.0,"[0.059680253, 0.012957457, 0.021607367, -0.097..."


In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop(["genre"], axis = 1)
y = data["genre"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.2)


In [ ]:
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, \
    precision_score, \
    recall_score, \
    ConfusionMatrixDisplay, \
    f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [5]:
from catboost import Pool

catboost_params = {
    'iterations': 3000,
    'learning_rate': 0.01,
    'eval_metric': 'Accuracy',
    'custom_metric': 'Precision',
    'early_stopping_rounds': 100,
    'use_best_model': True,
    'verbose': 50
}

train_pool = Pool(
    X_train,
    y_train,
    embedding_features = ['emb'],
    text_features = ['text'],
    cat_features = ["sign_count"]
)
valid_pool = Pool(
    X_test,
    y_test,
    embedding_features = ['emb'],
    text_features = ['text'],
    cat_features = ["sign_count"]
)

In [8]:
model = CatBoostClassifier(**catboost_params, random_state=42)
model.fit(train_pool, eval_set=valid_pool)
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_pred))
print(precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
print(recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
print(accuracy_score(y_true=y_test, y_pred=y_pred))
ConfusionMatrixDisplay.from_predictions(y_true=y_test, y_pred=y_pred)
plt.show()

In [ ]:
from pandas import to_pickle, read_pickle


to_pickle(model, "catboost_model")